# Data Prep for Canopy

## Create JSON File

Canopy reads local JSON / parquet files that contain the fields `["id", "text", "source", "metadata"]`. We will use the [`jamescalam/ai-arxiv`](https://huggingface.co/datasets/jamescalam/ai-arxiv) dataset. First we download it like so:

In [10]:
import os

os.environ["PINECONE_API_KEY"] = "81d500a8-2ab5-4737-a31c-80239252db14"
os.environ["OPENAI_API_KEY"] = "sk-proj-Ob6tB7BsBsr8FW4UbGT6T3BlbkFJqLnlAIlMfkg9dlmwN1Fh"
os.environ["INDEX_NAME"] = "test"

In [4]:
import pandas as pd

data = pd.read_json("data/test_data/train.jsonl", lines=True)

In [5]:
data

,id,title,summary,source,authors,categories,comment,journal_ref,primary_category,published,updated,content,references
0,2210.03945,Understanding HTML with Large Language Models,Large language models (LLMs) have shown except...,http://arxiv.org/pdf/2210.03945,"[Izzeddin Gur, Ofir Nachum, Yingjie Miao, Must...","[cs.LG, cs.AI]",None,None,cs.LG,20221008,20230519,UNDERSTANDING HTML WITH LARGE LANGUAGE\nMODELS...,"[{'id': '2107.06955'}, {'id': '2204.02311'}, {..."
1,1711.05101,Decoupled Weight Decay Regularization,L$_2$ regularization and weight decay regulari...,http://arxiv.org/pdf/1711.05101,"[Ilya Loshchilov, Frank Hutter]","[cs.LG, cs.NE, math.OC]",Published as a conference paper at ICLR 2019,None,cs.LG,20171114,20190104,Published as a conference paper at ICLR 2019\n...,"[{'id': '1707.08819'}, {'id': '1705.08292'}, {..."
2,2305.17493,The Curse of Recursion: Training on Generated ...,Stable Diffusion revolutionised image creation...,http://arxiv.org/pdf/2305.17493,"[Ilia Shumailov, Zakhar Shumaylov, Yiren Zhao,...","[cs.LG, cs.AI, cs.CL, cs.CR, cs.CV]",None,None,cs.LG,20230527,20230531,THECURSE OF RECURSION :\nTRAINING ON GENERATED...,"[{'id': '1906.02243'}, {'id': '1810.04805'}, {..."
3,2205.09712,Selection-Inference: Exploiting Large Language...,Large language models (LLMs) have been shown t...,http://arxiv.org/pdf/2205.09712,"[Antonia Creswell, Murray Shanahan, Irina Higg...","[cs.AI, cs.CL]",None,None,cs.AI,20220519,20220519,2022-5-20\nSelection-Inference: Exploiting Lar...,"[{'id': '2205.09712'}, {'id': '2112.03753'}, {..."
4,2104.06001,Gender Bias in Machine Translation,Machine translation (MT) technology has facili...,http://arxiv.org/pdf/2104.06001,"[Beatrice Savoldi, Marco Gaido, Luisa Bentivog...",[cs.CL],Accepted for publication in Transaction of the...,None,cs.CL,20210413,20210507,Gender Bias in Machine Translation\nBeatrice S...,"[{'id': '2104.06001'}, {'id': '2012.15859'}, {..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,2204.06745,GPT-NeoX-20B: An Open-Source Autoregressive La...,"We introduce GPT-NeoX-20B, a 20 billion parame...",http://arxiv.org/pdf/2204.06745,"[Sid Black, Stella Biderman, Eric Hallahan, Qu...",[cs.CL],To appear in the Proceedings of the ACL Worksh...,None,cs.CL,20220414,20220414,GPT-NeoX-20B: An Open-Source Autoregressive La...,"[{'id': '1910.05895'}, {'id': '2110.06609'}, {..."
419,1707.06347,Proximal Policy Optimization Algorithms,We propose a new family of policy gradient met...,http://arxiv.org/pdf/1707.06347,"[John Schulman, Filip Wolski, Prafulla Dhariwa...",[cs.LG],None,None,cs.LG,20170720,20170828,Proximal Policy Optimization Algorithms\nJohn ...,"[{'id': '1604.06778'}, {'id': '1506.02438'}, {..."
420,2307.09288,Llama 2: Open Foundation and Fine-Tuned Chat M...,"In this work, we develop and release Llama 2, ...",http://arxiv.org/pdf/2307.09288,"[Hugo Touvron, Louis Martin, Kevin Stone, Pete...","[cs.CL, cs.AI]",None,None,cs.CL,20230718,20230719,L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle : Open ...,"[{'id': '2302.13971'}, {'id': '1908.01091'}, {..."
421,2211.05100,BLOOM: A 176B-Parameter Open-Access Multilingu...,Large language models (LLMs) have been shown t...,http://arxiv.org/pdf/2211.05100,"[BigScience Workshop, :, Teven Le Scao, Angela...",[cs.CL],None,None,cs.CL,20221109,20230627,BLOOM: A 176B-Parameter Open-Access Multilingu...,"[{'id': '2208.07339'}, {'id': '2204.02311'}, {..."


In [6]:
len(data["content"][0])

66352

Then we must format it into the format we need (`["id", "text", "source", "metadata"]`):

In [7]:
# Function to map the DataFrame rows to the required format
def map_data(row):
    return {
        "id": row["id"],
        "text": row["content"],
        "source": row["source"],
        "metadata": {
            "title": row["title"],
            # "primary_category": row["primary_category"],
            # "published": row["published"],
            # "updated": row["updated"],
        },
    }
# Apply the function to the DataFrame
mapped_data = data.apply(map_data, axis=1).tolist()
df = pd.DataFrame(mapped_data)
df

,id,text,source,metadata
0,2210.03945,UNDERSTANDING HTML WITH LARGE LANGUAGE\nMODELS...,http://arxiv.org/pdf/2210.03945,{'title': 'Understanding HTML with Large Langu...
1,1711.05101,Published as a conference paper at ICLR 2019\n...,http://arxiv.org/pdf/1711.05101,{'title': 'Decoupled Weight Decay Regularizati...
2,2305.17493,THECURSE OF RECURSION :\nTRAINING ON GENERATED...,http://arxiv.org/pdf/2305.17493,{'title': 'The Curse of Recursion: Training on...
3,2205.09712,2022-5-20\nSelection-Inference: Exploiting Lar...,http://arxiv.org/pdf/2205.09712,{'title': 'Selection-Inference: Exploiting Lar...
4,2104.06001,Gender Bias in Machine Translation\nBeatrice S...,http://arxiv.org/pdf/2104.06001,{'title': 'Gender Bias in Machine Translation'}
...,...,...,...,...
418,2204.06745,GPT-NeoX-20B: An Open-Source Autoregressive La...,http://arxiv.org/pdf/2204.06745,{'title': 'GPT-NeoX-20B: An Open-Source Autore...
419,1707.06347,Proximal Policy Optimization Algorithms\nJohn ...,http://arxiv.org/pdf/1707.06347,{'title': 'Proximal Policy Optimization Algori...
420,2307.09288,L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle : Open ...,http://arxiv.org/pdf/2307.09288,{'title': 'Llama 2: Open Foundation and Fine-T...
421,2211.05100,BLOOM: A 176B-Parameter Open-Access Multilingu...,http://arxiv.org/pdf/2211.05100,{'title': 'BLOOM: A 176B-Parameter Open-Access...


In [9]:
df.to_json("data/test_data/ai_arxiv.jsonl", orient="records", lines=True)

## Jump into Canopy!

From here we can switch across to Canopy CLI (or other method):

First- modify canopy config:

1. Create config
```
canopy create-config
```
2. Edit Config File (default.yaml):
```
metric: cosine
spec:
  serverless:
    cloud: aws
    region: us-east-1
```
3. Set env variables:
```
export INDEX_NAME="test"
export PINECONE_API_KEY="..."
```
4. Run:
```
canopy new --config default.yaml
```

Now we can begin to work with out canopy framework:
```
canopy
```
Create and/or select index we want to use (or update the .env variable)
```
canopy new <index name>
```
```
canopy --index-name
```
```
canopy upsert ./RAG/data/test_data/ai_arxiv.jsonl
```

Then we begin chatting by first starting the Canopy Server:

```
canopy start
```

Then begin chatting with:

```
canopy chat
```

_(we can also add the `--no-rag` flag to see how our RAG vs. non-RAG results compare!)_